# NLP Core 3 Exercise: Bagged News

In this exercise we will learn how to perform document classification in order to predict the category of news articles from the Reuters Corpus using a **bag-of-words** model and **one-hot encoding**. We will then see how we can use **TF-IDF** to improve our features for classification.

## The Reuters Corpus

The Reuters Corpus is a collection of news documents along with category tags that are commonly used to test document classification. It is split into two sets: the *training* documents used to train a classification algorithm, and the *test* documents used to test the classifier's performance. Here we load the corpus and save the IDs of the training and test documents:

In [0]:
import nltk
nltk.download('reuters')
from nltk.corpus import reuters
train_ids = [d for d in reuters.fileids() if d.startswith('train')]
test_ids = [d for d in reuters.fileids() if d.startswith('test')]

[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!


**Questions**:
  1. How many documents are in the Reuters Corpus? What percentage are training and what percentage are testing documents?
  2. How many words are in the training documents? (use *reuters.words(file_id)*)

In [0]:
# answer to 1
n_total = len(train_ids + test_ids)
percent_training = round(len(train_ids) / n_total * 100)
print(len(train_ids + test_ids), 'documents in Reuters')
print(f'{percent_training}% training, {100 - percent_training}% testing')
# answer to 2
print(sum(len(reuters.words(train_id)) for train_id in train_ids), 'words in training documents')

10788 documents in Reuters
72% training, 28% testing
1253696 words in training documents


Let's have a look at the categories in the Reuters Corpus. Note that one document can have more than one category:

In [0]:
print(len(reuters.categories()), 'categories in the Reuters Corpus')
print('Categories of one sample document:', reuters.categories(train_ids[9]))
print('Sample from that document:', reuters.raw(train_ids[9])[:98])

90 categories in the Reuters Corpus
Categories of one sample document: ['cocoa', 'coffee', 'sugar']
Sample from that document: COFFEE, SUGAR AND COCOA EXCHANGE NAMES CHAIRMAN
  The New York Coffee, Sugar and Cocoa
  Exchange 


**Question:**
  3. What are the three most common categories in the training documents? (use *reuters.categories(file_id)*)

In [0]:
#answer to 3
from nltk import FreqDist
fd = FreqDist([category for train_id in train_ids for category in reuters.categories(train_id)])
print('most common 3 categories:', fd.most_common(3))

most common 3 categories: [('earn', 2877), ('acq', 1650), ('money-fx', 538)]


## Bag of words representations

We will now see how a sentence can be transformed into a feature vector using a bag of words model. Consider the following sentences:

In [0]:
sentences = [
  'This is the first document.',
  'This document is the second document.',
  'And this is the third one.',
   'Is this the first document?',
]

We can represent each word as a **one-hot** encoded vector (with a single 1 in the column for that word), and add their vectors together to get the feature vector for a sentence:

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sentences)
X.toarray()

array([[0, 1, 1, 1, 0, 0, 1, 0, 1],
       [0, 2, 0, 1, 0, 1, 1, 0, 1],
       [1, 0, 0, 1, 1, 0, 1, 1, 1],
       [0, 1, 1, 1, 0, 0, 1, 0, 1]], dtype=int64)

**Questions:**
  4. What do the rows and columns of the feature matrix X represent? **rows: sentences; columns: words**
  5. What word does the second column of X represent? What about the third column? (If you are stuck, look at *vectorizer.get_feature_names()*) **document; first**
 
 **Bonus**: Try using TfidfVectorizer instead of CountVectorizer, and try to explain why some values of X become smaller than others.

In [0]:
# answer for bonus question
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X2 = vectorizer.fit_transform(sentences)
X2.toarray()

array([[0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524],
       [0.        , 0.6876236 , 0.        , 0.28108867, 0.        ,
        0.53864762, 0.28108867, 0.        , 0.28108867],
       [0.51184851, 0.        , 0.        , 0.26710379, 0.51184851,
        0.        , 0.26710379, 0.51184851, 0.26710379],
       [0.        , 0.46979139, 0.58028582, 0.38408524, 0.        ,
        0.        , 0.38408524, 0.        , 0.38408524]])

## Classifying Reuters

Now let's put these together in order to build a classifier for Reuters articles. Fill in the following code using the instructions in the questions below:

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

train_docs = [reuters.raw(train_id) for train_id in train_ids]
test_docs = [reuters.raw(test_id) for test_id in test_ids]

In [0]:
#### (A) add code here from question 6
# answer:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(train_docs)
X2 = vectorizer.transform(test_docs)

In [0]:
# convert the category labels into binary features for classification
mlb = MultiLabelBinarizer()
y = mlb.fit_transform([reuters.categories(train_id) for train_id in train_ids])
y2 = mlb.transform([reuters.categories(test_id) for test_id in test_ids])

In [0]:
#### (B) add code here from question 7
# answer:
classifier = OneVsRestClassifier(LinearSVC())
classifier.fit(X, y)
predictions = classifier.predict(X2)

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [0]:
# show classifier's performance (look at average scores at the bottom)
print(classification_report(y2, predictions))

              precision    recall  f1-score   support

           0       0.97      0.95      0.96       719
           1       1.00      0.39      0.56        23
           2       1.00      0.64      0.78        14
           3       0.78      0.70      0.74        30
           4       0.92      0.67      0.77        18
           5       0.00      0.00      0.00         1
           6       1.00      0.83      0.91        18
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         3
           9       0.93      0.93      0.93        28
          10       1.00      0.78      0.88        18
          11       0.00      0.00      0.00         1
          12       0.91      0.86      0.88        56
          13       1.00      0.50      0.67        20
          14       0.00      0.00      0.00         2
          15       0.68      0.46      0.55        28
          16       0.00      0.00      0.00         1
          17       0.84    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)


**Questions:**
  6. In (A) above, add code to convert the training and testing documents into matrices X and X2 of feature vectors using CountVectorizer(). (Hint: use fit_transform() first on the training set, and then transform() on the testing set)
  7. In (B) above, add code to fit a multiclass SVM classifier on the training data . (Hint: use *OneVsRestClassifier(LinearSVC())* as the classifier object, and then call its fit() and predict() methods on the data.)
  
 **Bonus**: Try using TF-IDF (TfidfVectorizer) weighted features. Does the classifier's performance improve?

In [0]:
# answer to bonus question:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(train_docs)
X2 = vectorizer.transform(test_docs)
classifier = OneVsRestClassifier(LinearSVC(random_state=42))
classifier.fit(X, y)
predictions = classifier.predict(X2)
print(classification_report(y2, predictions))

              precision    recall  f1-score   support

           0       0.98      0.95      0.97       719
           1       1.00      0.39      0.56        23
           2       1.00      0.64      0.78        14
           3       0.94      0.53      0.68        30
           4       0.88      0.39      0.54        18
           5       0.00      0.00      0.00         1
           6       1.00      0.94      0.97        18
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         3
           9       0.96      0.96      0.96        28
          10       1.00      0.78      0.88        18
          11       0.00      0.00      0.00         1
          12       0.95      0.75      0.84        56
          13       1.00      0.50      0.67        20
          14       0.00      0.00      0.00         2
          15       0.92      0.39      0.55        28
          16       0.00      0.00      0.00         1
          17       0.90    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels.
  'precision', 'predicted', average, warn_for)
